In [1]:
import numpy as np
import pandas as pd
import mmlspark

# help(mmlspark)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
44,application_1523234702318_0014,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
data = spark.read.csv('/higgs/csv/HIGGS.csv', inferSchema=True, header=False)
data.show(10)

+---+------------------+-------------------+--------------------+------------------+--------------------+------------------+--------------------+--------------------+------------------+------------------+-------------------+--------------------+------------------+------------------+--------------------+-------------------+-----------------+-------------------+--------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+
|_c0|               _c1|                _c2|                 _c3|               _c4|                 _c5|               _c6|                 _c7|                 _c8|               _c9|              _c10|               _c11|                _c12|              _c13|              _c14|                _c15|               _c16|             _c17|               _c18|                _c19|                _c20|             _c21|              _c22|  

In [3]:
train, test = data.randomSplit([0.75, 0.25], seed=123)

In [4]:
train.rdd.getNumPartitions()

60

In [7]:
from timeit import default_timer as timer
start = timer()
from mmlspark import TrainClassifier
from mmlspark import LightGBMClassifier
model = TrainClassifier(model=LightGBMClassifier(numIterations=10, defaultListenPort=12450), labelCol="_c0", numFeatures=256).fit(train)
end = timer()
print(end - start)

98.0897130966

In [8]:
from timeit import default_timer as timer
start = timer()
from mmlspark import TrainClassifier
from pyspark.ml.classification import GBTClassifier
gbtmodel = TrainClassifier(model=GBTClassifier(), labelCol="_c0", numFeatures=256).fit(train)
end = timer()
print(end - start)

118.487487078

In [ ]:
gbtprediction = gbtmodel.transform(test)

In [ ]:
prediction = model.transform(test)

In [ ]:
from mmlspark import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric='AUC', labelCol='_c0', \
                                 scoredLabelsCol='scored_labels', scoresCol='scores').transform(prediction)
metrics.show()

In [ ]:
from mmlspark import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric='AUC', labelCol='_c0', \
                                 scoredLabelsCol='scored_labels', scoresCol='scored_labels').transform(gbtprediction)
metrics.show()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
gbtmetric = BinaryClassificationEvaluator(labelCol='_c0', metricName='areaUnderROC', rawPredictionCol="scored_labels").evaluate(gbtprediction)
gbtmetric

In [ ]:
prediction = model.transform(test)

In [ ]:
#prediction.select('_c0', 'scores', 'scored_probabilities', 'scored_labels').show(8, False)

In [ ]:
#validation.count()

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
metric = BinaryClassificationEvaluator(labelCol='_c0', metricName='areaUnderROC', rawPredictionCol="scores").evaluate(prediction)
metric

In [ ]:
from mmlspark import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric='AUC', labelCol='_c0', \
                                 scoredLabelsCol='scored_labels', scoresCol='scores').transform(prediction)
metrics.show()